# Department of Transportation - Oil Spill Mapping

In [ ]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import openpyxl
import folium
import json
import gmaps.datasets
import numpy as np
import re
from datetime import date, datetime
from django.core.serializers.json import DjangoJSONEncoder
from dataprep.clean import clean_lat_long
from dataprep.clean import validate_lat_long
# from shapely.geometry import Point, Polygon
# import geopandas as gpd
# import descartes
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import geopandas

%matplotlib inline
%matplotlib notebook

# Import API key
# from config import g_map

# Configure gmaps
# gmaps.configure(api_key=g_map)

## Bring in 2010 - Present Data

In [ ]:
# 1. Read the Data_Columns_and_spills into a DataFrame. All sheets
# Unedited excel files
xlfile_2010 = pd.ExcelFile("Resources/Data_columns_and_spills.xlsx", engine='openpyxl')
spill2010_df = xlfile_2010.parse('2010-Present') #'2010-Present' is the sheet name in the excel file
#spill2010_df is all data all columns

In [ ]:
spill2010_df.dtypes


In [ ]:
# formatting dataframe for mapping display. 
# Have to change datetime to string for json conversion
spill2010_df["REPORT_RECEIVED_DATE"] = spill2010_df["REPORT_RECEIVED_DATE"].dt.strftime('%Y-%m-%d %H:%M:%S')
spill2010_df["LOCAL_DATETIME"] = spill2010_df["LOCAL_DATETIME"].dt.strftime('%Y-%m-%d %H:%M:%S')
spill2010_df["NRC_RPT_DATETIME"] = spill2010_df["NRC_RPT_DATETIME"].dt.strftime('%Y-%m-%d %H:%M:%S')
spill2010_df["SHUTDOWN_DATETIME"] = spill2010_df["SHUTDOWN_DATETIME"].dt.strftime('%Y-%m-%d %H:%M:%S')
spill2010_df["RESTART_DATETIME"] = spill2010_df["RESTART_DATETIME"].dt.strftime('%Y-%m-%d %H:%M:%S')
spill2010_df["INCIDENT_IDENTIFIED_DATETIME"] = spill2010_df["INCIDENT_IDENTIFIED_DATETIME"].dt.strftime('%Y-%m-%d %H:%M:%S')
spill2010_df["ON_SITE_DATETIME"] = spill2010_df["ON_SITE_DATETIME"].dt.strftime('%Y-%m-%d %H:%M:%S')
spill2010_df["PREPARED_DATE"] = spill2010_df["PREPARED_DATE"].dt.strftime('%Y-%m-%d %H:%M:%S')
# Creating additional columns of UNINTENTIONAL_RELEASE_BBLS and PRPTY as a string for popup but still need 
# UNINTENTIONAL_RELEASE_BBLS and PRPTY as float for plotting
spill2010_df["UNINTENTIONAL_RELEASE_BBLS_STR"] = spill2010_df["UNINTENTIONAL_RELEASE_BBLS"].map("{:,.2f}".format)
spill2010_df["PRPTY_STR"] = spill2010_df["PRPTY"].map("${:,.0f}".format)



In [ ]:
spill2010_df["COMPONENT_AGE(YEARS)"] = (spill2010_df["IYEAR"] - spill2010_df["INSTALLATION_YEAR"]) + 1
spill2010_df.head()

In [ ]:
pipeline_file = pipeline_file.dropna(subset=["LONGITUDE"])

In [ ]:
pipeline_new = clean_lat_long(pipeline_file, lat_col="LATITUDE", long_col="LONGITUDE", split=False)
pipeline_new

In [ ]:
# Component age histrogram
plt.hist(spill2010_df['COMPONENT_AGE(YEARS)'], 20)

In [ ]:
plt.hist(spill2010_df['PRPTY'], 20)

In [ ]:
# Pulling useful columns for mapping purposes
my_columns = [
    'REPORT_NUMBER',
    'NAME',
    'LOCATION_LATITUDE',
    'LOCATION_LONGITUDE',
    'COMMODITY_RELEASED_TYPE',
    'UNINTENTIONAL_RELEASE_BBLS',
    'UNINTENTIONAL_RELEASE_BBLS_STR',
    'ON_OFF_SHORE',
    'ONSHORE_CITY_NAME',
    'ONSHORE_COUNTY_NAME',
    'ONSHORE_STATE_ABBREVIATION',
    'SYSTEM_PART_INVOLVED',
    'INSTALLATION_YEAR',
    'COMPONENT_AGE(YEARS)',
    'CAUSE',
    'ACCIDENT_PSIG',
    'MOP_PSIG',
    'PRPTY',
    'PRPTY_STR'
]

In [ ]:
# mapping columns called info2010_df
mapping_df = spill2010_df[my_columns]
# Creating another dataframe of first 100 rows called 'test' t2010_df
# Used to get map functioning and limiting API requests
# t2010_df = info2010_df[:100]
# t2010_df.to_json('test_df.json') # exports DataFrame to json for javascript and mapping functionality
# t2010_df
mapping_df

In [ ]:
mapping_df = mapping_df.fillna('N/A')
mapping_df

In [ ]:
mapping_dict = mapping_df.to_dict('records')
json_mapping = json.dumps(mapping_dict, indent = 4)
with open('mapping.json','w') as data:
    data.write(json_mapping)

In [ ]:
corrosion_df = mapping_df.loc[(mapping_df["CAUSE"] == "CORROSION FAILURE")]
corrosion_dict = corrosion_df.to_dict('records')
json_corrosion = json.dumps(corrosion_dict, indent = 4)
with open('corrosion.json','w') as data:
    data.write(json_corrosion)
    
corrosion_df.head(5)



In [ ]:
equipfail_df = mapping_df.loc[(mapping_df["CAUSE"] == "EQUIPMENT FAILURE")]
equipfail_dict = equipfail_df.to_dict('records')
json_equipfail = json.dumps(equipfail_dict, indent = 4)
with open('equipfail.json','w') as data:
    data.write(json_equipfail)
    
equipfail_df.head(5)


In [ ]:
excavation_df = mapping_df.loc[(mapping_df["CAUSE"] == "EXCAVATION DAMAGE")]
excavation_dict = excavation_df.to_dict('records')
json_excavation = json.dumps(excavation_dict, indent = 4)
with open('excavation.json','w') as data:
    data.write(json_excavation)
excavation_df.head(5)

In [ ]:
incorrect_df = mapping_df.loc[(mapping_df["CAUSE"] == "INCORRECT OPERATION")]
incorrect_dict = incorrect_df.to_dict('records')
json_incorrect = json.dumps(incorrect_dict, indent = 4)
with open('incorrect.json','w') as data:
    data.write(json_incorrect)
incorrect_df.head(5)

In [ ]:
matfail_df = mapping_df.loc[(mapping_df["CAUSE"] == "MATERIAL FAILURE OF PIPE OR WELD")]
matfail_dict = matfail_df.to_dict('records')
json_matfail = json.dumps(matfail_dict, indent = 4)
with open('matfail.json','w') as data:
    data.write(json_matfail)
matfail_df.head(5)

In [ ]:
natforce_df = mapping_df.loc[(mapping_df["CAUSE"] == "NATURAL FORCE DAMAGE")]
natforce_dict = natforce_df.to_dict('records')
json_natforce = json.dumps(natforce_dict, indent = 4)
with open('natforce.json','w') as data:
    data.write(json_natforce)
natforce_df.head(5)

In [ ]:
otherincd_df = mapping_df.loc[(mapping_df["CAUSE"] == "OTHER INCIDENT CAUSE")]
otherincd_dict = otherincd_df.to_dict('records')
json_otherincd = json.dumps(otherincd_dict, indent = 4)
with open('otherincd.json','w') as data:
    data.write(json_otherincd)
otherincd_df.head(5)

In [ ]:
otherforce_df = mapping_df.loc[(mapping_df["CAUSE"] == "OTHER OUTSIDE FORCE DAMAGE")]
otherforce_dict = otherforce_df.to_dict('records')
json_otherforce = json.dumps(otherforce_dict, indent = 4)
with open('otherforce.json','w') as data:
    data.write(json_otherforce)
otherforce_df.head(5)